In [3]:
using DrWatson
@quickactivate "GCN_HM_GRN-Integration"
using Turing, PyCall 

┌ Warning: Could not find find a project file by recursively checking given `path` and its parents. Returning `nothing` instead.
└ @ DrWatson /users/spate116/.julia/packages/DrWatson/bZN70/src/project_setup.jl:84


In [5]:
pushfirst!(PyVector(pyimport("sys")."path"), "");
train_model = pyimport("gcn").train_model
run_model = pyimport("gcn").run_model
extract_subgraph = pyimport("gcn").extract_subgraph
pyimport("sys")."stdout" = PyTextIO(stdout)
pyimport("sys")."stderr" = PyTextIO(stderr)

PyCall.PyError: PyError (PyImport_ImportModule

The Python package gcn could not be found by pyimport. Usually this means
that you did not install gcn in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/gpfs_home/spate116/ml/bin/python3

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the gcn module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-launch Julia.

Another alternative is to configure PyCall to use a Julia-specific Python
distribution via the Conda.jl package (which installs a private Anaconda
Python distribution), which has the advantage that packages can be installed
and kept up-to-date via Julia.  As explained in the PyCall documentation,
set ENV["PYTHON"]="", run Pkg.build("PyCall"), a

In [6]:
using Zygote
Zygote.@adjoint function pycall(f, x...; kw...)
    x = map(py, x)
    y = pycall(f, x...; kw...)
    y.detach().numpy(), function (ȳ)
        y.backward(gradient = py(ȳ))
        (nothing, map(x->x.grad.numpy(), x)...)
    end
end

ArgumentError: ArgumentError: Package Zygote not found in current path:
- Run `import Pkg; Pkg.add("Zygote")` to install the Zygote package.


In [7]:
node = 10
model, x, y, edge_index = train_model();
edge_index, node_idx = extract_subgraph(node, 2, edge_index)
edge_mask = repeat([1], get(edge_index."shape", 1));

UndefVarError: UndefVarError: train_model not defined

In [5]:
using Flux
Turing.setadbackend(:zygote)
@model graph_mask(y) = begin
    N = get(edge_index."shape", 1)
    edge_mask = Vector{Int}(undef, N)
    for i ∈ 1:N
        edge_mask[i] ~ Bernoulli(0.1)
    end
    pred = run_model(edge_mask, edge_index, model, node_idx)
    y = y/sum(y)
    y ~ MvNormal(pred/sum(pred), repeat([0.0001], length(y)))
end

DynamicPPL.ModelGen{var"###generator#427",(:y,),(),Tuple{}}(##generator#427, NamedTuple())

In [6]:
s = sample(graph_mask(run_model(repeat([1], get(edge_index."shape", 1)), edge_index, model, node_idx)), PG(2), 25000);

Sampling: 100%|█████████████████████████████████████████| Time: 0:02:20


In [7]:
describe(s)[1]

Summary Statistics
     parameters      mean       std   naive_se      mcse          ess      rhat  
         Symbol   Float64   Float64    Float64   Float64      Float64   Float64  
                                                                                 
   edge_mask[1]    0.1276    0.3336     0.0021    0.0208      56.5673    1.0977  
   edge_mask[2]    0.8714    0.3348     0.0021    0.0209      56.6888    1.0948  
   edge_mask[3]    0.4189    0.4934     0.0031    0.0311      51.4040    2.2274  
   edge_mask[4]    0.9751    0.1558     0.0010    0.0096      65.1074    1.0258  
   edge_mask[5]    0.4171    0.4931     0.0031    0.0311      51.1915    2.2548  
   edge_mask[6]    0.0004    0.0210     0.0001    0.0004    2512.5978    1.0004  
   edge_mask[7]    0.0001    0.0110     0.0001    0.0001   14650.9041    1.0001  
   edge_mask[8]    0.0076    0.0866     0.0005    0.0050     137.9516    1.0076  
   edge_mask[9]    0.3028    0.4595     0.0029    0.0290      51.5955    1.5195

In [8]:
ei = edge_index."detach"()."cpu"()."numpy"()

2×12 Array{Int64,2}:
  10    10  306  476  476   476   476   476  1140  1800  1986  2545
 476  2545  476   10  306  1140  1800  1986   476   476   476    10

In [9]:
base = run_model(repeat([1], get(edge_index."shape", 1)), edge_index, model, node_idx)
base/sum(base)

7-element Array{Float32,1}:
 0.00010267622
 0.1081773
 0.18511724
 0.18290088
 0.18526357
 0.17643231
 0.16200605

In [10]:
using DataFrames
ŷ = run_model([x > 0.1 ? 1 : 0 for x ∈ DataFrame(describe(s)[1])[!, 2]], edge_index, model, node_idx)
ŷ/sum(ŷ)

7-element Array{Float32,1}:
 0.00010563628
 0.10922247
 0.18864955
 0.1907632
 0.18037052
 0.16977596
 0.16111264

In [11]:
using GraphRecipes
using Plots
using LightGraphs

uni = unique(ei);
conv = Dict(uni[i] => i for i ∈ 1:length(uni));
conv_rev = Dict(i => uni[i] for i ∈ 1:length(uni));
vals = DataFrame(describe(s)[1])[!, 2];
weigh = Dict((ei[1, i], ei[2, i]) => vals[i] for i ∈ 1:length(vals))

Dict{Tuple{Int64,Int64},Float64} with 12 entries:
  (2545, 10)  => 0.871
  (476, 1140) => 0.00044
  (10, 2545)  => 0.87136
  (10, 476)   => 0.12756
  (476, 1800) => 0.00012
  (1986, 476) => 0.29608
  (1140, 476) => 0.30284
  (476, 10)   => 0.97512
  (306, 476)  => 0.41892
  (476, 306)  => 0.41712
  (1800, 476) => 0.00024
  (476, 1986) => 0.00756

In [ ]:
ei_conv = map(x -> conv[x], ei)
inp = ei_conv
classes = y."detach"()."cpu"()."numpy"()[unique(map(x -> conv_rev[x], inp))]

In [ ]:
g = DiGraph(Edge.(zip(inp[1, :], inp[2, :])))

graphplot(g, names=uni, arrow=true, nodecolor=classes, edgewidth=(s, d, w)-> weigh[(conv_rev[s],conv_rev[d])])
savefig("out.png")